In [1]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
from dataclasses import dataclass
from bisect import insort
import string

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [2]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

Expressions = {"humn": "humn"}
human = None
for line in lines:
    if line[:4] == "humn":
        human = int(line[6:])
        continue
    if len(line) < 17:
        Expressions[line[:4]] = line[6:]

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [3]:
class Monkey:

    def __init__(self, name, job):
        self.name = name
        self.job = job[5]
        self.left = job[:4]
        self.right = job[7:]
        self.expr = "{} {} {}".format(
            self.left, self.job, self.right)

    def perform(self, E):
        if self.left not in E or self.right not in E:
            return False

        expr = "({}) {} ({})".format(
            E[self.left], self.job, E[self.right])
        E[self.name] = expr
        self.expr = expr

        return True

    def __repr__(self):
        return f"{self.name}: {self.expr}"


Monkeys = [Monkey(line[:4], line[6:]) for line in lines if len(line) >= 17]


In [4]:
M = deepcopy(Monkeys)
E = deepcopy(Expressions)

while len(M) > 0:
    for m in M:
        if m.perform(E):
            M.remove(m)
            break

print(int(eval("lambda humn: " + E["root"])(human)))


38914458159166


### Part 2

In [5]:
from scipy.optimize import fsolve


root = next(line[6:] for line in lines if line[:4] == "root")
l, _, r = root.split()


left = eval("lambda humn: " + E[l])
right = eval("lambda humn: " + E[r])
x = int(fsolve(lambda h: left(h) - right(h),
        np.arange(-1_000_000, 1_000_000, 10_000))[0])


print(x)


3665520865940
